# Student Performance Factors Analysis

This notebook analyzes the `StudentPerformanceFactors.csv` dataset to predict exam scores using various machine learning models.

In [ ]:
# Install dependencies in the current Jupyter kernel
%pip install plotly pandas numpy scikit-learn xgboost seaborn nbformat

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

# Configuration
DATA_PATH = "StudentPerformanceFactors.csv"

## 1. Data Loading

In [ ]:
df = pd.read_csv(DATA_PATH)
print(f"Data shape: {df.shape}")
df.head()

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Summary Statistics
df.describe()

In [ ]:
# Distribution of Exam Score
fig = px.histogram(df, x='Exam_Score', nbins=30, title='Distribution of Exam Score', marginal='box')
fig.show()

In [ ]:
# Correlation Analysis (Numeric only)
numeric_df = df.select_dtypes(include=[np.number])
corr_matrix = numeric_df.corr()

fig = px.imshow(corr_matrix, 
                text_auto='.2f', 
                aspect="auto", 
                title='Correlation Matrix',
                color_continuous_scale='RdBu_r')
fig.show()

## 3. Data Preprocessing

In [ ]:
X = df.drop('Exam_Score', axis=1)
y = df['Exam_Score']

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

## 4. Model Training & Evaluation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
}

results = []
trained_models = {}

for name, model in models.items():
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    results.append({
        "Model": name,
        "MAE": mean_absolute_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R2 Score": r2_score(y_test, y_pred)
    })
    trained_models[name] = clf

results_df = pd.DataFrame(results).sort_values(by="RMSE", ascending=True)
results_df

In [ ]:
# Model Comparison Chart
fig = px.bar(results_df, x='Model', y='RMSE', color='R2 Score', 
             title='Model Performance Comparison (Lower RMSE is better)',
             text_auto='.3f')
fig.show()

## 5. Feature Importance

In [ ]:
best_model_name = results_df.iloc[0]['Model']
best_model = trained_models[best_model_name]

print(f"Extracting feature importance for best model: {best_model_name}")

preprocessor = best_model.named_steps['preprocessor']
num_features = preprocessor.transformers_[0][2]
cat_transformer = preprocessor.transformers_[1][1]
encoder = cat_transformer.named_steps['encoder']
cat_features_encoded = encoder.get_feature_names_out(preprocessor.transformers_[1][2])
feature_names = list(num_features) + list(cat_features_encoded)

regressor = best_model.named_steps['regressor']
if hasattr(regressor, 'feature_importances_'):
    importances = regressor.feature_importances_
elif hasattr(regressor, 'coef_'):
    importances = np.abs(regressor.coef_)

fi_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

fig = px.bar(fi_df.head(20), x='Importance', y='Feature', orientation='h',
             title=f'Top 20 Feature Importance - {best_model_name}',
             color='Importance')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()